In [1]:
import sys
!{sys.executable} -m pip install gmaps

## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

from config import g_key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kahului,US,20.8947,-156.4700,82.42,66,40,16.11,scattered clouds
1,1,Angoram,PG,-4.0625,144.0661,77.29,95,69,1.41,light rain
2,2,Ukiah,US,39.1502,-123.2078,70.16,39,0,9.22,clear sky
3,3,San Mateo Ixtatan,GT,15.8319,-91.4781,67.73,31,17,6.04,few clouds
4,4,Longkou,CN,37.6508,120.3286,52.39,59,100,6.44,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?70
What is the maximum temperature you would like for your trip?90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                      (city_data_df["Max Temp"] >= min_temp)]

In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             248
City                   248
Country                246
Lat                    248
Lng                    248
Max Temp               248
Humidity               248
Cloudiness             248
Wind Speed             248
Current Description    248
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.count()

Unnamed: 0             246
City                   246
Country                246
Lat                    246
Lng                    246
Max Temp               246
Humidity               246
Cloudiness             246
Wind Speed             246
Current Description    246
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kahului,US,82.42,scattered clouds,20.8947,-156.4700,
1,Angoram,PG,77.29,light rain,-4.0625,144.0661,
2,Ukiah,US,70.16,clear sky,39.1502,-123.2078,
9,Vaini,TO,81.23,overcast clouds,-21.2000,-175.2000,
10,Puerto Ayora,EC,78.76,light rain,-0.7393,-90.3518,
11,Hilo,US,80.28,very heavy rain,19.7297,-155.0900,
14,Saint George,US,84.79,clear sky,37.1041,-113.5841,
15,Tual,ID,84.33,overcast clouds,-5.6667,132.7500,
16,Butaritari,KI,81.63,few clouds,3.0707,172.7902,
20,Oussouye,SN,71.17,clear sky,12.4850,-16.5469,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df[hotel_df["Hotel Name"]!= '']
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kahului,US,82.42,scattered clouds,20.8947,-156.4700,Maui Seaside Hotel
10,Puerto Ayora,EC,78.76,light rain,-0.7393,-90.3518,Finch Bay Galapagos Hotel
14,Saint George,US,84.79,clear sky,37.1041,-113.5841,Best Western Plus Abbey Inn
16,Butaritari,KI,81.63,few clouds,3.0707,172.7902,Isles Sunset Lodge
23,San Carlos,VE,73.83,overcast clouds,9.6612,-68.5827,Hotel Paternopoli
...,...,...,...,...,...,...,...
671,Caravelas,BR,77.67,overcast clouds,-17.7125,-39.2481,Pousada dos Navegantes
672,Kununurra,AU,75.18,scattered clouds,-15.7667,128.7333,Hotel Kununurra
674,Marang,MY,79.30,broken clouds,5.2056,103.2059,DBarakah Homestay
678,Boma,GH,74.32,light rain,7.0805,-2.1697,Obaatanpa Hotel


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map.

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure

fig

Figure(layout=FigureLayout(height='420px'))